<a href="https://colab.research.google.com/github/abalckpie/professorahn/blob/main/%EB%B0%98%EB%B3%B5%EC%B8%A1%EC%A0%95_%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필요한 라이브러리 설치

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM

2. 데이터 준비

In [3]:
csv_test = pd.read_csv('/content/2022_2022년 지능정보사회 이용자 패널 조사_Data(가중치포함).csv')
data = csv_test.copy()
columns_of_interest = ['Age_group'] + [f'Q30_{i}' for i in range(1, 25)]
subset_data = data[columns_of_interest]

3. 그룹별 평균 계산

In [4]:
subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
subset_data['비판적 이해'] = subset_data[[f'Q30_{i}' for i in range(12, 16)]].mean(axis=1)
subset_data['생산과 공유'] = subset_data[[f'Q30_{i}' for i in range(16, 19)]].mean(axis=1)
subset_data['사회 참여'] = subset_data[[f'Q30_{i}' for i in range(19, 22)]].mean(axis=1)
subset_data['보안'] = subset_data[[f'Q30_{i}' for i in range(22, 25)]].mean(axis=1)

<ipython-input-4-50d5c1a80b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
<ipython-input-4-50d5c1a80b22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
<ipython-input-4-50d5c1a80b22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

4. 형식 변환(긴 형식, 범주형)

In [5]:
long_data = pd.melt(subset_data, id_vars=['Age_group'], value_vars=['생활활용', '권리보호', '비판적 이해', '생산과 공유', '사회 참여', '보안'],
                    var_name='Factor', value_name='Response')

In [6]:
long_data['Age_group'] = long_data['Age_group'].astype('category')

5. 데이터 수동 집계(피험자와 요인별 평균 계산)

In [8]:
agg_data = long_data.groupby(['Age_group', 'Factor']).mean().reset_index()

6. 반복측정 변량분석 수행 및 결과

In [9]:
aovrm = AnovaRM(agg_data, 'Response', 'Age_group', within=['Factor'])
res = aovrm.fit()

In [10]:
print(res)

               Anova
       F Value Num DF  Den DF Pr > F
------------------------------------
Factor 35.7822 5.0000 25.0000 0.0000



결과 해석 : F-value 값이 매우 크고(35.78) p-value가 0.0000으로 0.05보다 작기 때문에 6개 요인 간에 유의적인 차이가 있다.